**NOTE**: This is a Julia demo for Binder, it's been adapted from the Julia demo on the nbconvert website.

Binder is an open-source project and Julia support is early in development. If you'd like to improve Julia support in Binder, see our issue on adding Julia support here: https://github.com/jupyter/repo2docker/issues/23. We'd love to hear from you!

# An IJulia Demo

This notebook uses **IJulia**: a [Julia-language](http://julialang.org/) backend combined with the [IPython](http://ipython.org/) interactive environment.   This combination allows you to interact with the Julia language using IPython's powerful [graphical notebook](http://ipython.org/notebook.html), which combines code, formatted text, math, and multimedia in a single document.

## Basic Julia interaction

Basic mathematical expressions work like you expect:

In [ ]:
1 + sin(3)

You can define variables, write loops, and execute arbitrary multiline code blocks.  Here is an example of an alternating harmonic series $\sum_{n=1}^\infty \frac{(-1)^n}{n}$ from a [Julia tutorial by Homer Reid](http://homerreid.ath.cx/teaching/18.330/JuliaProgramming.shtml#SimplePrograms):

In [ ]:
s = 0.0
for n = 1:2:10000
    s += 1/n - 1/(n+1)
end
s # an expression on the last line (if it doesn't end with ";") is printed as "Out"

Previous outputs can be referred to via `Out[`*n*`]`, following the IPython, for example `Out[2]` for the result above.  You can also use the shorthand `_2`, or `_` for the previous result, as in IPython.  Like in Matlab, `ans` can also be used to refer to the previous result, *even if it was not printed* (when the command ended with `;`).

For example, the harmonic series above should be converging (slowly) to $\ln 2$, and we can check this:

In [ ]:
Out[2] - log(2)

Like Matlab or Scipy + Numpy, Julia has lots of mathematical functions and linear algebra built in.  For example, we can define a $500\times500$ random matrix $R$ and form the positive-definite matrix $R^* R$:

In [ ]:
R = rand(500,500)
R' * R

(Notice that, by default, only a portion of a large matrix is shown.  You didn't really want to read $500^2 = 250,000$ numbers, did you?)

Standard output from Julia is also captured and sent to the IJulia notebook as you'd expect:

In [ ]:
println("Hello world!\n")
println(stderr, "Börk börk börk, some unicode output on stderr...\n")

IJulia even captures output from external C libraries (and notice how easy it is using Julia's `ccall` intrinsic):

In [ ]:
ccall(:printf, Cint, (Ptr{UInt8},), "Hello from C!!\n");

We can define functions, of course, and use them in later input cells:

In [ ]:
f(x) = x .+ 1

In [ ]:
println(f(3))
f([1,1,2,3,5,8])

Notice that the input above both printed an scalar to `STDOUT` and also returned a vector, the latter using Julia's ability to write polymorphic functions and built-in array operations.

On the other hand adding a string to a number is not defined (there is no `+` method defined for those types, although we could easily add one), and attempting to do so will throw an exception:

In [ ]:
f("Hello?")

# Interactive Plotting in IJulia

Below we'll show off some plotting using the excellent Gadfly package. The plots are heavily inspired by [this blog post](http://int8.io/basic-visualization-in-julia-gadfly/).

In [ ]:
using RDatasets
sleep = dataset("lme4","sleepstudy")

In [ ]:
using Gadfly
plot(sleep, x = "Days", y = "Reaction", Geom.point)

In [ ]:
plot(sleep, x = "Reaction", Geom.density, color = "Subject", Scale.x_continuous(minvalue= 0, maxvalue= 500))

In [ ]:
plot(x = sleep[!, :Reaction], Geom.histogram(bincount = 30), Scale.x_continuous(minvalue = 200), color = sleep[!, :Days])

## Multimedia display in IJulia

Like most programming languages, Julia has a built-in `print(x)` function for outputting an object `x` as text, and you can override the resulting text representation of a user-defined type by overloading Julia's `show` function.  The next version of Julia, however, will extend this to a more general mechanism to display **arbitrary multimedia representations** of objects, as defined by standard [MIME types](https://en.wikipedia.org/wiki/Internet_media_type).   More specifically, the Julia [multimedia I/O API](http://docs.julialang.org/en/latest/stdlib/base/#multimedia-i-o) provides:

* A `display(x)` function requests the richest available multimedia display of a Julia object x (with a `text/plain` fallback).
* Overloading `writemime` allows one to indicate arbitrary multimedia representations (keyed by standard MIME types) of user-defined types.
* Multimedia-capable display backends may be registered by subclassing a generic `Display` type.  IJulia provides one such backend which, thanks to the IPython notebook, is capable of displaying HTML, LaTeX, SVG, PNG, and JPEG media formats.

The last two points are critical, because they separate **multimedia export** (which is defined by functions associated with the originating Julia data) from **multimedia display** (defined by backends which know nothing about the source of the data).

Precisely these mechanism were used to create the inline PyPlot plots above.   To start with, the simplest thing is to provide the MIME type of the data when you call `display`, which allows you to pass "raw" data in the corresponding format:

In [ ]:
display("text/html", """Hello <b>world</b> in <font color="red">HTML</font>!""")

However, it will be more common to attach this information to types, so that they display correctly automatically.  For example, let's define a simple `HTML` type in Julia that contains a string and automatically displays as HTML (given an HTML-capable backend such as IJulia):

In [ ]:
mutable struct HTML
   s::String
end

Base.show(io::IO, ::MIME"text/html", x::HTML) = print(io, x.s)

Here, `Base.show` is just a function that writes `x` in the corresponding format (`text/html`) to the I/O stream `io`.

This `show` definition is all that we need to make any object of type `HTML` display automatically as HTML text in IJulia:

In [ ]:
x = HTML("<ul> <li> Hello from a bulleted list! </ul>")

In [1]:
display(x)
println(x)

LoadError: UndefVarError: x not defined

In [3]:
] add CSV

   Resolving package versions...
  No Changes to `~/Project.toml`
  No Changes to `~/Manifest.toml`


In [4]:
] dataframes

LoadError: `dataframes` is not a recognized command. Type ? for help with available commands

In [5]:
] add DataFrames

   Resolving package versions...
    Updating `~/Project.toml`
  [a93c6f00] + DataFrames v1.2.2
  No Changes to `~/Manifest.toml`


In [6]:
using CSV
using DataFrames

In [7]:
alldata = CSV.read("data.csv", DataFrame)

LoadError: ArgumentError: "data.csv" is not a valid file

In [8]:
size(alldata)

LoadError: UndefVarError: alldata not defined

In [ ]:
data = [5:end, :]; # take from the fifth row, since the first four is wonky. since it's 2-dimensional, need to give two ranges ";" hides output

In [9]:
using WGLMakie # plotting

LoadError: ArgumentError: Package WGLMakie not found in current path:
- Run `import Pkg; Pkg.add("WGLMakie")` to install the WGLMakie package.


In [ ]:
f = Figure()
Axis(f[1,1])
errorbars!(data.x, data.y, data.sigma_y) # plots the error bars of the points
scatter!(data.x, data.y, markersize = 20, color:= red)
f

In [10]:
b_eye = 50.
m_eye = 2. # eyeballing the slope and intercept

50

In [ ]:
xx = LinRange(50, 250, 20) # range of x that we want a plot

In [ ]:
f = Figure()
Axis(f[1,1])
errorbars!(data.x, data.y, data.sigma_y) # plots the error bars of the points
scatter!(data.x, data.y, markersize = 20, color:= red)

yy_eye = xx .* m_eye .+ b_eye
lines!(xx, yy_eye)

f

In [ ]:
] add Optim

In [ ]:
using Optim

In [ ]:
# optimize m amd b values. We define what's best, and let the function to find the best m and b in all values of m and b
function objective(paramters) # the function that we want to optimize
    b = parameters[1]
    m = parameters[2]
    @show b m
    return abs(b-3.4) + abs(m-17)
end

In [ ]:
starting_params = [b,_eye, m_eye]
result = optimize(objective, starting_params) # call the function to be optimized, pass in starting parameters
# it will find the values that minimize the returned value from objective



In [ ]:
function objective_2(paramters) # the function that we want to optimize
    b = parameters[1]
    m = parameters[2]
    y_pred = b .+ m .* data.x
    return sum(abs.(data.y .- y_pred)) # without sum, it returns a vector
end

In [ ]:
starting_params = [b,_eye+0., m_eye]
result = optimize(objective_2, starting_params) # call the function to be optimized, pass in starting parameters
# it will find the values that minimize the returned value from objective
Optim.minimizer(result) # pull out the optimal values


In [ ]:
function objective_abs(paramters, x, y, sigma) # the function that we want to optimize
    b = parameters[1]
    m = parameters[2]
    
    y_pred = b .+ m .* x
    ## optimize using gaussian distribution??
    return -sum( -log(sigma*sqrt(2*\pi)) .-0.5 .* (y .- y_pred).^2 /sigma.^2) # without sum, it returns a vector
end

In [ ]:
# function shim(p)
#     return objective_abs(p, data.x, data.y, data.sigma_y)
# end

In [ ]:
# anonymous functions
starting_params = [b_eye + 0., m_eye]
result = optimize(p -> objective_abs(p, data.x, data.y, data.sigma_y), starting_params)
